In [ ]:
# adapter-transformers library is based on torch==1.4.0 and transformers==3.1.0.
!pip install torch==1.4.0
!pip transformers==3.1.0
!pip install adapter-transformers==1.1.1

     |████████████████████████████████| 753.4 MB 7.0 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
ERROR: unknown command "transformers==3.1.0"
     |████████████████████████████████| 1.3 MB 7.9 MB/s 
     |████████████████████████████████| 2.9 MB 50.2 MB/s 
     |████████████████████████████████| 1.1 MB 56.6 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 


## Import required Python libraries

In [ ]:
import torch, os
import pandas as pd
import torch.nn as nn
from tqdm import tqdm, trange

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaConfig, RobertaModelWithHeads, RobertaTokenizer, AdapterType, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score


## Set seed of randomization and working device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)


def set_seed(seed):
    # Set the random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

cuda


### Define data generator class and preparation function.

The custom dataset should inherit [`Dataset`](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) and define the following methods:
  * `__len__` so that len(dataset) returns the size of the dataset.
  * `__getitem__` to support the indexing such that `dataset[i]` can be used to get $i$th sample

In [ ]:
class CustomDataset(Dataset):
    # initialization
    def __init__(self, dataframe, tokenizer, max_len, lab2ind):
        """
          dataframe: pandas DataFrame.
          tokenizer: Hugginfance BERT/RoBERTa tokenizer
          max_len: maximal length of input sequence
          lab2ind: dictionary of label classes
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = self.data.content
        self.labels = self.data.label
        self.max_len = max_len
        self.lab2ind = lab2ind

    # get the size of the dataset
    def __len__(self):
        return len(self.comment_text)

    # generate sample by index
    def __getitem__(self, index):
        # get ith sample and label
        comment_text = str(self.comment_text[index])
        label = str(self.labels[index])

        label = self.lab2ind[label]
        # use encode_plus() of Transformers to tokenize and vectorize input seuqnce and covert it to tensors. 
        # this method truncate or pad sequence to the maximal length and then return pytorch tensors. 
        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_tensors = "pt"
        )

        return {
            'ids': inputs['input_ids'],
            'mask': inputs['attention_mask'],
            'targets': torch.tensor(label, dtype=torch.long)
        }

### Define a function to load datasets and create data iterators.

In [ ]:
def regular_encode(file_path, tokenizer, lab2ind, shuffle=True, num_workers = 2, batch_size=64, maxlen = 32, mode = 'train'): 
    '''
      file_path: path to your dataset file
      tokenizer: tokenizer method
      lab2ind: label-to-index dictionary
      shuffle: shuffle the dataset or not
      num_workers: a number of data processors
      batch_size: the number of batch size
      maxlen: maximal sequence length
      mode: the type of dataset
    '''
    # if we are in train mode, we will load two columns (i.e., text and label).
    if mode == 'train':
        # Use pandas to load dataset, the dataset should be a tsv file where the first line is the header.
        df = pd.read_csv(file_path, delimiter='\t',header=0, names=['content','label'], encoding='utf-8', quotechar=None, quoting=3)
    
    # if we are in predict mode, we will load one column (i.e., text).
    elif mode == 'predict':
        df = pd.read_csv(file_path, delimiter='\t',header=0, names=['content', 'label'])
    else:
        print("the type of mode should be either 'train' or 'predict'. ")
        return
        
    print("{} Dataset: {}".format(file_path, df.shape))
    # instantiate the dataset instance 
    custom_set = CustomDataset(df, tokenizer, maxlen,lab2ind)
    
    dataset_params = {'batch_size': batch_size, 'shuffle': shuffle, 'num_workers': num_workers}

    batch_data_loader = DataLoader(custom_set, **dataset_params)
    # return a data iterator
    return batch_data_loader

### Training and evaluation functions.

In [ ]:
def train(model, iterator, optimizer, scheduler, criterion):
    
    model.train()
    epoch_loss = 0.0
    
    for _, batch in enumerate(iterator):
        # load data batch
        input_ids = batch['ids'].to(device, dtype = torch.long)
        input_mask = batch['mask'].to(device, dtype = torch.long)
        labels = batch['targets'].to(device, dtype = torch.long)
        # forward
        outputs = model(input_ids, input_mask, labels=labels)
        loss, logits = outputs[:2]
        
        # delete used variables to free GPU memory
        del batch, input_ids, input_mask, labels
        optimizer.zero_grad()
        # backward
        if torch.cuda.device_count() == 1:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            epoch_loss += loss.cpu().item()
        else:
            loss.mean().backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            epoch_loss += loss.mean().cpu().item()

        optimizer.step()
        scheduler.step()
    
    # free GPU memory
    if device == 'cuda':
        torch.cuda.empty_cache()

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    all_pred=[]
    all_label = []

    with torch.no_grad():
        for _, batch in enumerate(iterator, 0):
        # Add batch to GPU
            input_ids = batch['ids'].to(device, dtype = torch.long)
            input_mask = batch['mask'].to(device, dtype = torch.long)
            labels = batch['targets'].to(device, dtype = torch.long)
            # forward
            outputs = model(input_ids, input_mask, labels=labels)
            loss, logits = outputs[:2]

            # delete used variables to free GPU memory
            del batch, input_ids, input_mask

            if torch.cuda.device_count() == 1:
                epoch_loss += loss.cpu().item()
            else:
                epoch_loss += loss.sum().cpu().item()
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(logits.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    # computing metrics 
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    recall = recall_score(all_label, all_pred, average='macro')
    precision = precision_score(all_label, all_pred, average='macro')

    return epoch_loss/len(iterator), accuracy, f1score, recall, precision

### Create a optimizer and scheduler.

The model train with a linear learing rate [scheduler](https://huggingface.co/transformers/main_classes/optimizer_schedules.html#transformers.get_linear_schedule_with_warmup) that decreases linearly from the peak learning rate to 0 after a warmup period where learning rate linearly increase from 0 to the peak learning rate. 

![](https://huggingface.co/transformers/_images/warmup_linear_schedule.png)



In [ ]:
def create_optimizer_and_scheduler(model, num_training_steps, warmup_steps, learning_rate):
    """
    Setup the optimizer and the learning rate scheduler.
    num_training_steps: the number of training steps
    warmup_steps: the number of warm-up steps
    learning_rate: the peak learning rate
    """
    optimizer = AdamW(
    model.parameters(),
    lr=learning_rate
    )
    
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=warmup_steps, 
        num_training_steps=num_training_steps
    )

    return optimizer, lr_scheduler

### Train Adapter


Load model and tokenizer of `RoBERTa-Base` by the shortcut name.

In [ ]:

model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModelWithHeads.from_pretrained(model_name)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The roberta-base has {count_parameters(model):,} trainable parameters')

The roberta-base has 124,645,632 trainable parameters


Add task-specific adapter module for sociality classification task that is a single text classification task. By calling `train_adapter(["social"])`, we freeze all transformer parameters and only optimize the parameters of `social` adapter.

In [ ]:
model.add_adapter("social", AdapterType.text_task)
model.train_adapter(["social"])

Add the classification head, i.e., a two-layer feed-forward neural network, on top of the Transformer layers. 

The method `model.set_active_adapters([["social"]])` registers the `social` adapter as a default for training. 

In [ ]:
model.add_classification_head("social", num_labels=2)
model.set_active_adapters([["social"]])

Send model to device (CPU/GPU).

In [ ]:
model = model.to(device)

In [ ]:
print(model)

RobertaModelWithHeads(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
print(f'The social adapter RoBERTa model has {count_parameters(model):,} trainable parameters')

The social adapter RoBERTa model has 1,486,658 trainable parameters


The number of trainable parameters decrease significantly.

### Training

Specify hyper-parameters and load datasets.

We freeze all the Transformer layers and only optimize the parameters of adapter modules that are new added and randomly initialized. Hence, we use large learning rate (i.e., 3e-4).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/585data/train.csv", delimiter=";")
train_data = train_data[["text", "account.type"]]
train_data.to_csv("/content/drive/MyDrive/585data/train.tsv", sep="\t", encoding="utf-8", index=False)

In [ ]:
dev_data = pd.read_csv("/content/drive/MyDrive/585data/validation.csv", delimiter=";")
dev_data = train_data[["text", "account.type"]]
dev_data.to_csv("/content/drive/MyDrive/585data/dev.tsv", sep="\t", encoding="utf-8", index=False)

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/585data/test.csv", delimiter=";")
test_data = train_data[["text", "account.type"]]
test_data.to_csv("/content/drive/MyDrive/585data/test.tsv", sep="\t", encoding="utf-8", index=False)

In [ ]:
lab2ind = {'human': 0, 'bot': 1}
batch_size = 32
max_seq_length = 32
num_epochs = 5
warmup_proportion = 0.1
learning_rate = 3e-4
max_grad_norm = 1.0
data_dir = "/content/drive/MyDrive/585data/"

train_file = os.path.join(data_dir, "train.tsv")
dev_file = os.path.join(data_dir, "dev.tsv")
test_file = os.path.join(data_dir, "test.tsv")

Create data interators. 

In [ ]:
train_dataloader = regular_encode(train_file, tokenizer, lab2ind, shuffle=True, batch_size=batch_size, maxlen = max_seq_length)
validation_dataloader = regular_encode(dev_file, tokenizer, lab2ind, shuffle=False, batch_size=batch_size, maxlen = max_seq_length)
test_dataloader = regular_encode(test_file, tokenizer, lab2ind, shuffle=False, batch_size=batch_size, maxlen = max_seq_length)


/content/drive/MyDrive/585data/train.tsv Dataset: (22590, 2)
/content/drive/MyDrive/585data/dev.tsv Dataset: (22590, 2)
/content/drive/MyDrive/585data/test.tsv Dataset: (22590, 2)


Optimizer, sheduler, and loss function.

In [ ]:
num_training_steps	= len(train_dataloader) * num_epochs
num_warmup_steps = num_training_steps * warmup_proportion

In [ ]:
optimizer, scheduler = create_optimizer_and_scheduler(model, num_training_steps, num_warmup_steps, learning_rate)
criterion = nn.CrossEntropyLoss()

Train the model with 10 epochs. The training speed is much faster than fully fine-tuning (i.e., optimize the parameters of the entire RoBERTa). We save the `social` adapter module at the end of each epoach rather than the entire RoBERTa model. This `social` adapter is light weight: it is only 3MB! 

In [ ]:
epoch_res = []

for epoch in trange(num_epochs, desc="Epoch"):
    train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
    val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)

    epoch_eval_result = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
                      "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1
                      }
    print(epoch_eval_result)
    epoch_res.append(epoch_eval_result)
    save_path = "./epoch"+str(epoch+1) 
    if os.path.exists(save_path) == False:
      os.makedirs(save_path)

    model.save_all_adapters(save_path)


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]


KeyError: ignored

Present the validation preformance.

In [ ]:
report_df = pd.DataFrame(epoch_res)
report_df.sort_values(by=["val_f1"], ascending=False, inplace=True)
report_df

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1
1,2,0.132116,0.941288,0.943161,0.939930,0.940966
2,3,0.119514,0.938447,0.940198,0.937072,0.938101
3,4,0.110955,0.938447,0.940198,0.937072,0.938101
4,5,0.110976,0.938447,0.940198,0.937072,0.938101
0,1,0.145735,0.936553,0.938286,0.935173,0.936196


### Load the best adapter model and evaluate on Test set

Load the pre-trained RoBERTa model by shortcut name. 

In [ ]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModelWithHeads.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Load the trained task-specific adapter module that achieves the best performance on validation set.  

In [ ]:
adapter_name = model.load_adapter("./epoch2/social")

Overwriting existing adapter 'social'.
Overwriting existing head 'social'


Add the trained adapter to RoBERTa and evaluate on test set.

In [ ]:
model.set_active_adapters(adapter_name)
model = model.to(device)
test_loss, test_acc, test_f1, test_recall, test_precision = evaluate(model, test_dataloader, criterion)

In [ ]:
print(test_loss, test_acc, test_f1, test_recall, test_precision)

0.2498893676833673 0.9204545454545454 0.9199480181936321 0.9219626778024272 0.918901412100013


### References:
* Houlsby, N., Giurgiu, A., Jastrzebski, S., Morrone, B., De Laroussilhe, Q., Gesmundo, A., ... & Gelly, S. (2019, May). [Parameter-efficient transfer learning for NLP](http://proceedings.mlr.press/v97/houlsby19a/houlsby19a.pdf). In International Conference on Machine Learning (pp. 2790-2799). PMLR.

* https://docs.adapterhub.ml/index.html

* https://medium.com/dair-ai/adapters-a-compact-and-extensible-transfer-learning-method-for-nlp-6d18c2399f62
